# 파일명만 바꿔준 뒤 모두 실행하시면 됩니다.
### df_summary['누락_주차'][0] 로 어떤 주차가 누락인지 확인가능

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import date
import os

In [31]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [32]:
file_path = 'C:\\ai_x\\source\\JikFam\\data\\배추_이상치제거_주간기준_등급코드.csv'
output_dir = 'C:\\ai_x\\source\\JikFam\\eda_results'
os.makedirs(output_dir, exist_ok=True)

In [33]:
try:
    df = pd.read_csv(file_path, encoding='cp949')
except FileNotFoundError:
    print(f"오류: 파일 '{file_path}'을(를) 찾을 수 없습니다.")
    raise

In [34]:
print("--- 기초 통계량 ---")
print(df.describe())
print("--- 데이터 정보 ---")
df.info()

--- 기초 통계량 ---
           품목코드           품종코드           등급코드        총금액(원)       총거래량(kg)  \
count  983690.0  983690.000000  983690.000000  9.836900e+05  983690.000000   
mean        1.0      31.034062      12.201084  2.022335e+06    2690.909213   
std         0.0      41.308382       0.879732  8.511261e+06    9999.330792   
min         1.0       0.000000      11.000000  1.000000e+02       0.050000   
25%         1.0       4.000000      11.000000  1.002000e+05     100.000000   
50%         1.0       8.000000      12.000000  3.870000e+05     410.000000   
75%         1.0      99.000000      13.000000  1.386000e+06    1940.000000   
max         1.0      99.000000      20.000000  7.100456e+08  784070.000000   

             평균단가(원)      주간평균단가(원)         직팜산지코드  휴일여부  명절지수  작기정보  \
count  983690.000000  983690.000000  983690.000000   0.0   0.0   0.0   
mean     1263.126936    1262.368560    1068.441629   NaN   NaN   NaN   
std      1189.237995     594.748315      64.940992   NaN   NaN   N

In [35]:
print("--- 결측치 확인 ---")
print(df.isnull().sum())

--- 결측치 확인 ---
주차                   0
연월일                  0
품목코드                 0
품목명                  0
품종코드                 0
품종명                  0
등급코드                 0
등급이름                 0
총금액(원)               0
총거래량(kg)             0
평균단가(원)              0
주간평균단가(원)            0
직팜산지코드               0
휴일여부            983690
명절지수            983690
작기정보            983690
일평균기온             2310
최고기온              2310
최저기온              2310
평균상대습도            2310
강수량(mm)           2310
1시간최고강수량(mm)      2310
dtype: int64


In [36]:
df['주차시작일'] = df['주차'].str.extract(r'^(\d{4}-\d{2}-\d{2})')
df['주차시작일'] = pd.to_datetime(df['주차시작일'], format='%Y-%m-%d', errors='coerce')

iso = df['주차시작일'].dt.isocalendar()
df['주차_연도'] = iso.year
df['주차_번호'] = iso.week

In [37]:
df[['주차', '주차시작일', '주차_연도', '주차_번호']].head()

,주차,주차시작일,주차_연도,주차_번호
0,2018-01-01~2018-01-07,2018-01-01,2018,1
1,2018-01-01~2018-01-07,2018-01-01,2018,1
2,2018-01-01~2018-01-07,2018-01-01,2018,1
3,2018-01-01~2018-01-07,2018-01-01,2018,1
4,2018-01-01~2018-01-07,2018-01-01,2018,1


In [45]:
from datetime import date

def get_weeks_in_year(year):
    return date(year, 12, 28).isocalendar()[1]

summary = []

for crop in df['품목명'].unique():
    df_crop = df[df['품목명'] == crop].copy()

    df_crop = df_crop[(df_crop['주차_연도'] >= 2018) & (df_crop['주차_연도'] <= 2024)]

    actual_weeks = set(zip(df_crop['주차_연도'], df_crop['주차_번호']))

    expected_weeks = set()
        for w in range(1, get_weeks_in_year(y) + 1):
            expected_weeks.add((y, w))

    missing_weeks = sorted(list(expected_weeks - actual_weeks))

    null_rows = df_crop.isnull().any(axis=1).sum()
    outlier_rows = df_crop[
        (df_crop['평균단가(원)'] > 6000) | (df_crop['평균단가(원)'] <= 0)
    ].shape[0]

    summary.append({
        '품목명': crop,
        '존재_연도범위': "2018~2024",  # 🔒 고정
        '예상_주차수': len(expected_weeks),
        '실제_주차수': len(actual_weeks),
        '누락_주차수': len(missing_weeks),
        '결측치_행수': null_rows,
        '단가_이상치_행수': outlier_rows,
        '누락_주차': missing_weeks
    })

df_summary = pd.DataFrame(summary)

In [46]:
df_summary[['품목명', '예상_주차수', '실제_주차수', '누락_주차수']]

,품목명,예상_주차수,실제_주차수,누락_주차수
0,배추,365,365,0


In [47]:
len(missing_weeks)

0

In [48]:
def format_weeks(week_list):
    return [f"{y}년도 {w}주차" for (y, w) in week_list]

summary = []

for crop in df['품목명'].unique():
    df_crop = df[df['품목명'] == crop].copy()

    is_outlier = (df_crop['평균단가(원)'] > 6000) | (df_crop['평균단가(원)'] <= 0)
    df_outlier = df_crop[is_outlier]

    outlier_weeks = sorted(set(zip(df_outlier['주차_연도'], df_outlier['주차_번호'])))
    outlier_weeks_str = format_weeks(outlier_weeks)

    summary.append({
        '품목명': crop,
        '이상치_주차수': len(outlier_weeks),
        '이상치_주차': outlier_weeks_str
    })

df_outlier_summary = pd.DataFrame(summary)

In [44]:
df_outlier_summary[['품목명', '이상치_주차수']]

,품목명,이상치_주차수
0,배추,147
